In [ ]:
## set covid19_datasciente as path for python find bulletin package
import sys
from pathlib import Path
from os import getcwd, remove, walk
from os.path import join, basename, isdir
sys.path.append(str(Path(getcwd()).parent.parent))

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
# pd.options.display.float_format = lambda x: '{:,.0f}'.format(x).replace(',','.')

import codecs
import numpy as np
from glob import glob
from epiweeks import Week

In [ ]:
from bulletin import default_input, default_output, tables_path, root
default_output = join(default_output,'vacinacao')

from bulletin.systems import Notifica, CasosConfirmados, eSUS, Sivep
from bulletin.utils.static import Municipios
from bulletin.utils.xls_ import fit_cols
from bulletin.utils.normalize import normalize_text, normalize_hash, date_hash, normalize_cpf

from datetime import datetime, date, timedelta

hoje = pd.to_datetime(date.today())
ontem = hoje - timedelta(1)
anteontem = ontem - timedelta(1)
data_retroativos = ontem - timedelta(31)

tables = dict([(Path(x).stem,pd.read_csv(join(tables_path,x))) for x in glob(join(tables_path,"*.csv"))])

In [ ]:
situacao_labels = { 0:'Não Vacinado', 1:'Esquema Primário Incompleto', 2:'Esquema Primário Completo ', 3: 'Esquema Primário Completo + Reforço'}
fator_risco_labels = { 0:'Sem Fator de Risco', 1:'Com Fator de Risco' }
mes_labels = {key+1:value for key,value in enumerate(['Janeiro','Fevereiro','Março','Abril','Maio','Junho','Julho','Agosto','Setembro','Outubro','Novembro','Dezembro'])}
evolucao_labels = {1: 'Recuperados', 2: 'Óbitos', 3: 'Ativos'}

In [ ]:
from difflib import SequenceMatcher

def similar(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [ ]:
vac = eSUS('vacinados')
vac.load()
vac.df.shape

In [ ]:
vacinados = vac.df

In [ ]:
autistas = pd.read_excel(join(default_input,'autistas.xlsx'))
autistas

In [ ]:
autistas['hash_mae'] = autistas['Nome Requerente'].apply(normalize_hash) + autistas['Nome mãe'].apply(normalize_hash)
autistas['hash_nasc'] = autistas['Nome Requerente'].apply(normalize_hash) + autistas['Data Nascimento'].apply(date_hash)
autistas['cpf'] = autistas['CPF Requerente'].apply(normalize_cpf)

In [ ]:
casos_vacinados_cpf = pd.merge(vacinados.loc[vacinados['cpf'].notna()],autistas, on='cpf', how='inner',suffixes=['_esus','_autistas'])
print(len(casos_vacinados_cpf))

In [ ]:
casos_vacinados_mae = pd.merge(vacinados.loc[vacinados['hash_mae'].notna()],autistas, on='hash_mae', how='inner',suffixes=['_esus','_autistas'])
print(len(casos_vacinados_mae))

In [ ]:
casos_vacinados_nasc = pd.merge(vacinados.loc[vacinados['hash_nasc'].notna()],autistas, on='hash_nasc', how='inner',suffixes=['_esus','_autistas'])
print(len(casos_vacinados_nasc))

In [ ]:
casos_vacinados = pd.concat([casos_vacinados_cpf,casos_vacinados_mae,casos_vacinados_nasc], ignore_index=True)
print(len(casos_vacinados))

In [ ]:
casos_vacinados.columns

In [ ]:
casos_vacinados.groupby('cns')[['paciente']].count().sort_values('paciente',ascending=False)

In [ ]:
new_df = pd.DataFrame()

for cns, df in casos_vacinados.groupby('cns'):
    new_df = new_df.append(df.apply(lambda x: x.loc[x.notna()].unique(), axis=0).apply(lambda x: x[0] if (len(x) >= 1) else None), ignore_index=True)

new_df

In [ ]:
new_df.pivot_table(index=['situacao_atual'], values='cns', aggfunc='count', margins=True, margins_name='Total').rename(index=situacao_labels, level='situacao_atual').rename(columns={'cns':'quantidade'})

In [ ]:
autistas = autistas.join(new_df.drop_duplicates('Numero Carteira',keep='first').set_index('Numero Carteira')[[col for col in new_df.columns if not col in autistas.columns]], on='Numero Carteira')

In [ ]:
autistas.pivot_table(index=['situacao_atual'], values='Numero Carteira', aggfunc='count', margins=True, margins_name='Total', dropna=False).rename(index=situacao_labels, level='situacao_atual').rename(columns={'cns':'quantidade'})

In [ ]:
autistas.loc[autistas['situacao_atual'].isna(),'situacao_atual'] = 0
autistas.groupby('situacao_atual')[['Numero Carteira']].count().rename(index=situacao_labels, level='situacao_atual').rename(columns={'Numero Carteira':'quantidade'})

In [ ]:
autistas.replace({'situacao_atual':situacao_labels}).to_excel(join(default_output,'autistas.xlsx'))